In [1]:
import torch
import helper
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Implementing th neural network of the paper pieces by pieces

## Convolution block

In [73]:
class ConvBNReluBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, resampling_filter=None, padding=1, upsample=False, downsample=False):
        super().__init__()
        
        ### Convolution layer
        
        self.block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, 
                      out_channels=out_channels, 
                      kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        
        self.upsample = upsample
        self.downsample = downsample
        
        self.resampling_filter = resampling_filter
        
    def forward(self, x):
        x = self.block(x)
        
        if self.downsample:
            x = F.dropout2d(x, p=0.05)
            x = F.max_pool2d(x, kernel_size=self.resampling_filter)
        
        if self.upsample:
            x = F.interpolate(x, mode='nearest', scale_factor=self.resampling_filter)
        
        return x

## Encoder part

In [74]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.cbe1 = ConvBNReluBlock(1, 32, kernel_size=(3, 2), resampling_filter=(5, 2), downsample=True)
        self.cbe2 = ConvBNReluBlock(32, 128, kernel_size=(3, 2), resampling_filter=(4, 2), downsample=True)
        self.cbe3 = ConvBNReluBlock(128, 256, kernel_size=(3, 2), resampling_filter=(2, 2), downsample=True)
        
    def forward(self, x):
        x = self.cbe1(x)
        x = self.cbe2(x)
        x = self.cbe3(x)
        
        return x

In [75]:
Encoder()(torch.zeros((1, 1, 1000, 8))).shape

torch.Size([1, 256, 25, 1])

# Residual block

source: https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/

In [76]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        
        self.convbnrelu1 = ConvBNReluBlock(in_channels, in_channels, kernel_size = 3)
        
        self.convbnrelu2 = ConvBNReluBlock(in_channels, in_channels, kernel_size = 3)
        
        self.convbnrelu3 = ConvBNReluBlock(in_channels, in_channels, kernel_size = 3)
        
    def forward(self, x):
        residual = x
        
        x = self.convbnrelu1(x) 
        x = self.convbnrelu2(x)
        x = self.convbnrelu3(x)
        
        return x + residual

In [77]:
class ResNet(nn.Module):
    def __init__(self, in_channels, num_blocks):
        super(ResNet, self).__init__()
        self.blocks = [ResidualBlock(in_channels=256) for i in range(num_blocks)]
    
    def forward(self, x):
        for block in self.blocks:
            x = block(x)
            
        return x

In [78]:
encoder = Encoder()
resnet = ResNet(in_channels=256, num_blocks=5)

In [79]:
resnet(encoder(torch.zeros((1, 1, 1000, 8)))).shape

torch.Size([1, 256, 25, 1])

## Decoder part

In [104]:
# Add upsample to the decoder, not correct yet
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convbnrelu1 = ConvBNReluBlock(256, 128, kernel_size=(3, 2), resampling_filter=(5, 3), padding=1, upsample=True)
        self.convbnrelu2 = ConvBNReluBlock(128, 32, kernel_size=(3, 2), resampling_filter=(4, 1), padding=1, upsample=True)
        self.convbnrelu3 = ConvBNReluBlock(32, 1, kernel_size=(3, 2), resampling_filter=(2, 2), padding=1, upsample=True)
        
    def forward(self, x):
        x = self.convbnrelu1(x)
        x = self.convbnrelu2(x)
        x = self.convbnrelu3(x)
        
        return x

In [105]:
encoder = Encoder()
resnet = ResNet(in_channels=256, num_blocks=5)
decoder = Decoder()

In [106]:
decoder(resnet(encoder(torch.zeros((2, 1, 1000, 8))))).shape

torch.Size([2, 1, 1000, 16])